In [1]:
from getpass import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
if "SERPERDEV_API_KEY" not in os.environ:
    os.environ["SERPERDEV_API_KEY"] = getpass("Enter SerperDev API key: ")

### Simple LLM response with web search tool

In [2]:
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.websearch import SerperDevWebSearch
from haystack.dataclasses import ChatMessage
from haystack.tools.component_tool import ComponentTool

# Create a web search tool using SerperDevWebSearch
web_tool = ComponentTool(component=SerperDevWebSearch(), name="web_tool")

# Create the agent with the web search tool
agent = Agent(chat_generator=OpenAIChatGenerator(model="gpt-4o-mini"), tools=[web_tool])

# Run the agent with a query
result = agent.run(messages=[ChatMessage.from_user("Find information about Haystack AI framework")])

# Print the final response
print(result["messages"][-1].text)

Here are some key resources and information about the Haystack AI framework:

1. **Official Website**: [Haystack by Deepset](https://haystack.deepset.ai/)
   - Haystack is a production-ready open-source AI framework designed for building customizable applications powered by Large Language Models (LLMs) and other AI tools.

2. **GitHub Repository**: [deepset-ai/haystack](https://github.com/deepset-ai/haystack)
   - This repository contains the code and documentation to get started with the Haystack framework, which supports building applications using LLMs, transformer models, and vector search.

3. **Documentation**: [Haystack Documentation](https://docs.haystack.deepset.ai/docs/intro)
   - Comprehensive documentation is available to help users understand how to use the framework effectively.

4. **Tutorial on Building AI Applications**: [DataCamp Tutorial](https://www.datacamp.com/tutorial/haystack-ai-tutorial)
   - This tutorial introduces how to utilize Haystack for building AI appl

In [3]:
from haystack.components.generators.utils import print_streaming_chunk

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4o-mini"), tools=[web_tool], streaming_callback=print_streaming_chunk
)

result = agent.run(messages=[ChatMessage.from_user("Find information about Haystack AI framework")])


[TOOL CALL]
Tool: web_tool 
Arguments: {"query":"Haystack AI framework"}

[TOOL RESULT]
{'documents': [{'title': 'Haystack | Haystack', 'link': 'https://haystack.deepset.ai/', 'sitelinks': [{'title': 'Overview', 'link': 'https://haystack.deepset.ai/overview/intro'}, {'title': 'Documentation', 'link': 'https://docs.haystack.deepset.ai/docs/intro'}], 'position': 1, 'id': '1104f63a625503314beb6ecab3ee3994e52ebd7be29f6674719c7331a7249b82', 'content': 'The Production-Ready Open Source AI Framework · Highly customizable · Build with leading LLM providers and AI tools · Production is where it gets real.', 'blob': None, 'score': None, 'embedding': None, 'sparse_embedding': None}, {'title': 'deepset-ai/haystack - GitHub', 'link': 'https://github.com/deepset-ai/haystack', 'position': 2, 'id': 'e45b44b66370e7c22ce22ace37c78c16aad632ac8905966b30487c8d62be22db', 'content': 'Haystack is an end-to-end LLM framework that allows you to build applications powered by LLMs, Transformer models, vector sear

### Pipeline to tool

In [4]:
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.converters.html import HTMLToDocument
from haystack.components.converters.output_adapter import OutputAdapter
from haystack.components.fetchers.link_content import LinkContentFetcher
from haystack.components.websearch.serper_dev import SerperDevWebSearch
from haystack.dataclasses import ChatMessage
from haystack.core.pipeline import Pipeline

search_pipeline = Pipeline()

search_pipeline.add_component("search", SerperDevWebSearch(top_k=10))
search_pipeline.add_component("fetcher", LinkContentFetcher(timeout=3, raise_on_failure=False, retry_attempts=2))
search_pipeline.add_component("converter", HTMLToDocument())
search_pipeline.add_component(
    "output_adapter",
    OutputAdapter(
        template="""
{%- for doc in docs -%}
  {%- if doc.content -%}
  <search-result url="{{ doc.meta.url }}">
  {{ doc.content|truncate(25000) }}
  </search-result>
  {%- endif -%}
{%- endfor -%}
""",
        output_type=str,
    ),
)

search_pipeline.connect("search.links", "fetcher.urls")
search_pipeline.connect("fetcher.streams", "converter.sources")
search_pipeline.connect("converter.documents", "output_adapter.docs")


🚅 Components
  - search: SerperDevWebSearch
  - fetcher: LinkContentFetcher
  - converter: HTMLToDocument
  - output_adapter: OutputAdapter
🛤️ Connections
  - search.links -> fetcher.urls (list[str])
  - fetcher.streams -> converter.sources (list[ByteStream])
  - converter.documents -> output_adapter.docs (list[Document])

In [5]:
from haystack.core.super_component import SuperComponent
from haystack.tools import ComponentTool
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator

search_component = SuperComponent(
    pipeline=search_pipeline,
    input_mapping={"query": ["search.query"]},
    output_mapping={"output_adapter.output": "search_result"},
)

search_tool = ComponentTool(
    name="search",
    description="Use this tool to search for information on the internet.",
    component=search_component,
    outputs_to_string={"source": "search_result"},
)

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4o-mini"),
    tools=[search_tool],
    system_prompt="""
    You are a deep research assistant.
    You create comprehensive research reports to answer the user's questions.
    You use the 'search'-tool to answer any questions.
    You perform multiple searches until you have the information you need to answer the question.
    Make sure you research different aspects of the question.
    Use markdown to format your response.
    When you use information from the websearch results, cite your sources using markdown links.
    It is important that you cite accurately.
    """,
    exit_conditions=["text"],
    max_agent_steps=20,
)


In [6]:
query = "What are the latest updates on the Artemis moon mission?"
messages = [ChatMessage.from_user(query)]

agent.warm_up()
agent_output = agent.run(messages=messages)

print(agent_output["messages"][-1].text)

### Latest Updates on the Artemis Moon Mission (as of October 2023)

The Artemis program, led by NASA, is set to return humans to the Moon for the first time since the Apollo missions. Below are key highlights and updates regarding the Artemis II mission and subsequent missions.

#### Artemis II Mission Overview
- **Mission Crew**: Artemis II will carry four astronauts: NASA Commander Reid Wiseman, Pilot Victor Glover, Mission Specialist Christina Koch, and Canadian Space Agency (CSA) astronaut Jeremy Hansen.
- **Scheduled Launch**: The mission is planned to launch in a window between **February and April 2026**.
- **Flight Path**: Artemis II will undertake a **10-day mission around the Moon** without landing, which will be the first crewed mission to orbit the Moon since 1972.
- **Spacecraft**: The astronauts will travel in the **Orion spacecraft**, launched atop the **Space Launch System (SLS) rocket**.

#### Mission Objectives
- **Testing Systems**: The flight aims to test the space

In [7]:
from IPython.display import Markdown, display

display(Markdown(agent_output["messages"][-1].text))

### Latest Updates on the Artemis Moon Mission (as of October 2023)

The Artemis program, led by NASA, is set to return humans to the Moon for the first time since the Apollo missions. Below are key highlights and updates regarding the Artemis II mission and subsequent missions.

#### Artemis II Mission Overview
- **Mission Crew**: Artemis II will carry four astronauts: NASA Commander Reid Wiseman, Pilot Victor Glover, Mission Specialist Christina Koch, and Canadian Space Agency (CSA) astronaut Jeremy Hansen.
- **Scheduled Launch**: The mission is planned to launch in a window between **February and April 2026**.
- **Flight Path**: Artemis II will undertake a **10-day mission around the Moon** without landing, which will be the first crewed mission to orbit the Moon since 1972.
- **Spacecraft**: The astronauts will travel in the **Orion spacecraft**, launched atop the **Space Launch System (SLS) rocket**.

#### Mission Objectives
- **Testing Systems**: The flight aims to test the spacecraft's life support systems and other critical technologies needed for future lunar landings.
- **Scientific Objectives**: The crew will collect scientific data and conduct observations of the Moon. They may see parts of the far side of the Moon that no humans have observed closely before.

#### Hardware and Preparations
- **Rocket Stacking**: The SLS rocket is nearly fully stacked at NASA's Kennedy Space Center, with essential hardware arriving and being integrated into the launch vehicle.
- **Stage Adapter Delivery**: The final piece of the Artemis II rocket, the **stage adapter**, has been successfully delivered and integrated, enabling the connection of the Orion spacecraft with the SLS rocket.
  
#### Recent Milestones
- **Crew Training**: The astronaut team has been engaged in extensive training, which includes simulations and emergency response exercises. They recently practiced mission scenarios, including re-entry procedures and responding to system failures.
- **Heat Shield Inspection**: Following issues identified during the Artemis I mission, the heat shield of the Orion spacecraft has been modified and is now deemed ready for the upcoming flight, with a new trajectory designed to mitigate previous issues.

#### Future Missions
- **Artemis III**: Looking ahead, Artemis III is scheduled for **no earlier than mid-2027** and aims to land astronauts on the lunar surface, marking the first crewed Moon landing since Apollo 17 in 1972. This mission will be crucial for establishing a sustainable human presence on the Moon.
  
#### Broader Significance
The Artemis program signifies a new era of lunar exploration, with goals extending beyond just landing astronauts on the Moon. The missions are designed to pave the way for future expeditions to Mars and foster international collaborations for space exploration.

NASA emphasizes the importance of Artemis II as a stepping stone for human exploration, aiming to learn about living and working in lunar environments and to gather data on the effects of deep space radiation on humans.

For more detailed updates, you can follow NASA's official blog [here](https://www.nasa.gov/blogs/artemis/).

--- 

These updates reflect the most current information available regarding the Artemis moon missions as of October 2023.